In [1]:
import numpy as np
import matplotlib.pyplot as plt
import legwork as lw
import astropy.units as u
import tqdm
from astropy.cosmology import Planck18, z_at_value
from scipy.integrate import trapz
from schwimmbad import MultiPool

In [2]:
def get_peak_fGW(e, forb):
    sum = -1.01678 * e + 5.57372 * e**2 - 4.9271 * e**3 + 1.68506*e**4
    npeak = 2 * 1 + sum * (1-e**2)**(7/2)

    return forb * npeak

In [3]:
e_grid = np.logspace(-3, np.log10(0.999), 10)
mass1_grid = np.logspace(np.log10(5), np.log10(50), 10)
mass2_grid = np.logspace(np.log10(5), np.log10(50), 10)
f_grid = np.logspace(-4, -1, 100)

In [ ]:
M1, M2, E, F = np.meshgrid(mass1_grid, mass2_grid, e_grid, f_grid)

mass_ratio_mask = M1 > M2

M1 = M1[mass_ratio_mask]
M2 = M2[mass_ratio_mask]
E = E[mass_ratio_mask]
F = F[mass_ratio_mask]
FPEAK = get_peak_fGW(E, F)

In [ ]:
fpeak_mask = FPEAK > 1e-1

M1 = M1[fpeak_mask]
M2 = M2[fpeak_mask]
E = E[fpeak_mask]
F = F[fpeak_mask]

print(F)


In [ ]:
source = lw.source.Source(m_1=M1 * u.Msun,
                          m_2=M2 * u.Msun,
                          ecc=E,
                          f_orb=F*u.Hz,
                          dist=8 * np.ones(len(E)) * u.Mpc,
                          interpolate_g=True)

In [ ]:
snr = source.get_snr(verbose=True)

KeyboardInterrupt: 

In [ ]:
with MultiPool(processes=2) as pool:
    snr = np.array(list(pool.map(integrate_de_dt, zip(ecc_i, timesteps.tolist(), beta, c_0))))

In [ ]:

m1_list = []
m2_list = []
e_list = []
f_list = []
snrs = []
for m1 in tqdm.tqdm(mass1_grid):
    for m2 in mass2_grid:
        if m2 < m1:
            for e in e_grid:                 
                e_evol = np.ones(len(f_grid)) * e
                m1_evol = np.ones(len(f_grid)) * m1
                m2_evol = np.ones(len(f_grid)) * m2
                f_evol = f_grid
                
                source = lw.source.Source(m_1=m1_evol * u.Msun,
                                          m_2=m2_evol * u.Msun,
                                          ecc=e_evol,
                                          f_orb=f_evol*u.Hz,
                                          dist=8 * np.ones(len(e_evol)) * u.Mpc,
                                          interpolate_g=False)
                                              
                snr = source.get_snr(approximate_R=True, verbose=False)

                snrs.extend(snr)         
                source = []
                snr = []

In [ ]:
%%prun -s cumulative -q -l 20 -T prun0

source = lw.source.Source(m_1=np.ones(len(e_grid)) * m1 * u.Msun,
                          m_2=np.ones(len(e_grid)) * m2 * u.Msun,
                          ecc=e_grid,
                          f_orb=np.ones(len(e_grid))*f*u.Hz,
                          dist=8 * np.ones(len(e_grid)) * u.Mpc,
                          interpolate_g=True,
                          n_proc=1)
                          
snr = source.get_snr(approximate_R=True, verbose=True)


In [ ]:
print(open('prun0', 'r').read())



In [ ]:

snrs = []
for f in tqdm.tqdm(f_grid):
    source = lw.source.Source(m_1=np.array(m1_list)*u.Msun,
                              m_2=np.array(m2_list)*u.Msun,
                              ecc=np.array(e_list),
                              f_orb=np.ones(len(m1_list))*f*u.Hz,
                              dist=8 * np.ones(len(m1_list)) * u.Mpc,
                              interpolate_g=True,
                              n_proc=36)
                              
    snr = source.get_snr(approximate_R=True, verbose=True)
    snrs.append(snr)                          